# Imports

In [1]:
! pip install keras-tuner
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as Layer
import numpy as np
import keras_tuner as kt
import tensorflow.keras.layers.experimental.preprocessing as preprocessing
import statistics


In [2]:
import re
import pandas as pd

movieFields = ['id', 'title', 'vote_average', 'vote_count', 'genres']
castFields = ['id', 'cast']

movieData = pd.read_csv('movies_metadata.csv', skipinitialspace=True, usecols=movieFields, engine='python')
castData = pd.read_csv('credits.csv', skipinitialspace=True, usecols=castFields, engine='python')

dfMovies = pd.DataFrame(movieData)
dfCast = pd.DataFrame(castData)

df = pd.merge(dfMovies, dfCast, how='inner')


# ---------------------------------------------------------------------------- #
# The following section formats the cast and creates a clean list of actor names.

unformattedNames = df['cast']

# main list of cast grouped by movies
cast_master_copy = []

# list of overall cast by individual names
actor_name_list = []

# reads through the cast string for each movie
for unformattedNameString in unformattedNames:

    # creates new list for current movie
    grouped = ""

    # pulls actors' names out of the paragraph
    line_list = re.findall("(?<=\'name\': )(.*?)(?=,)", unformattedNameString)
    
    # converts list of cast for one movie into a string to manipulate further 
    line_string = str(line_list)

    # separates cast into a unique list per movie
    line_split = line_string.split(", ")

    actor_count = 0
    # reads through each actor per movie
    for name in line_split:
        if (actor_count == 5):
            break
     
        # removes extraneous symbols from the actors' names
        name = name.replace("\"", "")
        name = name.replace("\'", "")
        name = name.replace("[", "")
        name = name.replace("]", "")
        name = name.replace(" ", "")

        # adds actors to two working lists
        # grouped: actors are in lists by movies
        # actor_name_list: puts every actors in one list
        grouped += name + ' '
        actor_name_list.append(name)
        
        actor_count += 1

    # adds formatted cast members to list
    grouped = grouped[:-1]
    cast_master_copy.append(grouped)

# assigns formatted cast to 'cast' column of DataFrame
df['cast'] = cast_master_copy
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section formats the genre and adds it to the data frame

unformatted_genres = df['genres']

# main list of genres grouped by movies
genre_master_copy = []

# list of overall cast by individual names
genre_name_list = []

# reads through the genre string for each movie
for unformatted_genre_string in unformatted_genres:

    # creates new list for current movie
    genre_grouped = []

    # pulls genres out of the paragraph
    genre_line_list = re.findall("(?<=\'name\': )(.*?)(?=})", unformatted_genre_string)

    # converts list of genres for one movie into a string to manipulate further 
    genre_line_string = str(genre_line_list)

    # separates genre into a unique list per movie
    genre_line_split = genre_line_string.split(", ")

    # reads through each genre per movie & grabs the first one
    for genre in genre_line_split:
     
        # removes extraneous symbols from the genres
        genre = genre.replace("\"", "")
        genre = genre.replace("\'", "")
        genre = genre.replace("[", "")
        genre = genre.replace("]", "")

        genre_str = str(genre)
        break

    # adds formatted genre to list
    genre_master_copy.append(genre_str)

# assigns formatted cast to 'cast' column of DataFrame
df['genres'] = genre_master_copy
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section appends the genre to the cast string. This preparation is 
# needed for text vectorization / embedding.

final_format_col = []
cast = df['cast'].to_list()
genres = df['genres'].to_list()

i = 0
count = len(cast)
for i in range(count):
    final_string = cast[i] + " " + genres[i]
    final_format_col.append(final_string)

df.insert(loc=0, column='training_col', value=final_format_col)
df.training_col = final_format_col
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section filters out data points that will be harmful for
# our model's accuracy and drops unnecessary columns.

# gets rid of all movies with 0 votes
# Number of movies before operation: 43019
# Number of movies after operation: 22651
df = df.drop(df[df.vote_count < 10].index)
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section handles the creation of the .csv files that will be the
# primary data used to train the models. 

# writes formatted DataFrame values to a new csv
pd.DataFrame.to_csv(df, "formatted_movies.csv", index="false")

# creates a csv of the actor dictionary
# w = csv.writer(open("actor_dict.csv", "w"))
# for key, val in actor_dict.items():
#     w.writerow([key, val])
# ---------------------------------------------------------------------------- #

np.random.seed(1)
tf.random.set_seed(1)

movie_df = df
movie_df.drop(['vote_count'], axis = 1)

data_count = movie_df.shape[0]
training_data_count = int(data_count * 0.75) + 1
test_data_count = data_count - training_data_count

y_train = np.array(movie_df['vote_average'].head(training_data_count))
x_train = np.array(movie_df['training_col'].head(training_data_count))

y_test = np.array(movie_df['vote_average'].tail(test_data_count))
x_test = np.array(movie_df[['training_col']].tail(test_data_count))

std = statistics.stdev(movie_df['vote_average'].to_list())
mean = statistics.mean(movie_df['vote_average'].to_list())
variance = statistics.variance(movie_df['vote_average'].to_list())
mode = statistics.mode(movie_df['vote_average'].to_list())


VOCAB_SIZE = 201277
encoder = preprocessing.TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=6)
features = np.array(df['training_col'])
encoder.adapt(features)

# Feed Forward Network

In [6]:
!rm -rf untitled_project

def build_model(hp):
  
  model = tf.keras.Sequential()
  model.add(encoder)
  
  model.add(Layer.Embedding(
            input_dim=len(encoder.get_vocabulary()),
            # output_dim=hp.Int('embedding_neurons', min_value=400, max_value=500, step_size=50)
            output_dim=500)
            
            ))

  model.add(Layer.Flatten(input_shape=(16989, 1)))

  for i in range(hp.Int('num_hidden_layers', 1, 4)):
    model.add(Layer.Dense(hp.Int(f'dense{i}_neurons', min_value=0, max_value=100, step=20), activation='relu'))

  model.add(Layer.Dense(1, name='output'))

  # optimizer=hp.Choice('optimizer', values=['RMSprop'])
  optimizer = 'RMSprop'
  
  # tf.keras.optimizers.Adam(hp.Choice('learning_rate', [.01, .001, .0001]))
  tf.keras.optimizers.Adam(hp.Choice(.001)
  

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=optimizer,
              metrics=['mse'])
  
  return model


tuner = kt.RandomSearch(
build_model,
objective='val_loss',
max_trials=300,  # how many model variations to test?
executions_per_trial=1)  # how many trials per variation? (same model could perform differently

tuner.search_space_summary(5)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 6)                 0         
_________________________________________________________________
embedding (Embedding)        (None, 6, 450)            18569250  
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 200)               130200    
_________________________________________________________________
dense (Dense)                (None, 60)                12060     
_________________________________________________________________
output (Dense)               (None, 1)                 61        
Total params: 18,711,571
Trainable params: 18,711,571
Non-trainable params: 0
_________________________________________________________________
Search space summary
Default search space size: 5
embedding_neurons (Int)
{'default': None, 'conditions': [], 

## FFNN Training & Validation

In [ ]:
tuner.search(x=x_train,
            y=y_train,    
            verbose=2,
            epochs=1,
            batch_size=512,
            callbacks=[],
            validation_data=(x_test, y_test))

tuner.results_summary()

### Keras-Tuner Hyperparameter Optimization
### batch size 512 unless otherwise specified
Go in Editing Mode to View

# Keras Tuner Findings

# General performance test 1
general test with many values to see how the model reacts to different nums of neurons

### Search Space

Default search space size: 4
embedding_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 450, 'max_value': 550, 'step': 50, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 400, 'step': 50, 'sampling': None}
{'default': 'adam', 'conditions': [], 'values': ['adam', 'RMSprop'], 'ordered': False}
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


### Results

Trial 74 Complete [00h 00m 06s]

Best val_loss So Far: 0.8339388370513916
Total elapsed time: 00h 11m 06s

Hyperparameters:
embedding_neurons: 500
dense1_neurons: 250
optimizer: RMSprop
learning_rate: 0.001
Score: 0.8339388370513916

Hyperparameters:
embedding_neurons: 500
dense1_neurons: 250
optimizer: RMSprop
learning_rate: 0.01
Score: 0.8427369594573975

Hyperparameters:
embedding_neurons: 550
dense1_neurons: 300
optimizer: RMSprop
learning_rate: 0.01
Score: 0.844895601272583

Hyperparameters:
embedding_neurons: 550
dense1_neurons: 250
optimizer: RMSprop
learning_rate: 0.001
Score: 0.8483849763870239

Hyperparameters:
embedding_neurons: 500
dense1_neurons: 250
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.8536829948425293

# General performance test 2
general test with many values to see how the model reacts to different nums of neurons

### Search Space
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 210, 'step': 50, 'sampling': None}
{'default': 'adam', 'conditions': [], 'values': ['adam', 'RMSprop'], 'ordered': False}
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
### Results

Trial 132 Complete [00h 00m 21s]

Best val_loss So Far: 0.7398236989974976
Total elapsed time: 01h 28m 48s

Search: Running Trial #133

Hyperparameter    |Value             |Best Value So Far 
embedding_neurons |500               |500               
num_hidden_layers |5                 |5                 
dense0_neurons    |110               |60                
optimizer         |RMSprop           |RMSprop           
learning_rate     |0.0001            |0.0001            
dense1_neurons    |160               |160               
dense2_neurons    |160               |160               
dense3_neurons    |160               |210               
dense4_neurons    |160               |160  











# Embedding Neurons trial 1 SELECTED 500
only changing the num of embedding neurons to find the best . looking from 100-1000 broadly

batch size 512
### Search Space
Search space summary
Default search space size: 1
embedding_neurons (Int) {'default': None, 'conditions': [], 'min_value': 100, 'max_value': 1000, 'step': 100, 'sampling': None}
### Results

Trial 8 Complete [00h 00m 02s]

Best val_loss So Far: 0.8410091996192932
Total elapsed time: 00h 00m 58s
Showing 10 best trials
Objective(name='val_loss', direction='min')

Trial summary
Hyperparameters:
embedding_neurons: 500
Score: 0.8410091996192932

Trial summary
Hyperparameters:
embedding_neurons: 400
Score: 0.8689964413642883

Trial summary
Hyperparameters:
embedding_neurons: 700
Score: 0.8894386887550354

Trial summary
Hyperparameters:
embedding_neurons: 300
Score: 0.91794753074646

Trial summary
Hyperparameters:
embedding_neurons: 800
Score: 0.926380455493927



# Embedding neurons trial 2
narrowing down on embedding neurons testing 400-700 step size 50

### Search Space
Default search space size: 1
embedding_neurons (Int) {'default': None, 'conditions': [], 'min_value': 400, 'max_value': 700, 'step': 50, 'sampling': None}
### Results
Trial 5 Complete [00h 00m 11s]

Best val_loss So Far: 0.8405100107192993
Total elapsed time: 00h 00m 40s
INFO:tensorflow:Oracle triggered exit
Results summary
Hyperparameters:

embedding_neurons: 600
Score: 0.8405100107192993

Trial summary
Hyperparameters:
embedding_neurons: 400
Score: 0.8456131815910339

Trial summary
Hyperparameters:
embedding_neurons: 650
Score: 0.8536071181297302

Trial summary
Hyperparameters:
embedding_neurons: 450
Score: 0.8710063695907593

Trial summary
Hyperparameters:
embedding_neurons: 550
Score: 0.8793086409568787



# Embedding neurons trial 4
narrowing down on embedding neurons testing 500-800 step size 20

batch size 512
### Search Space
Search space summary
Default search space size: 1
embedding_neurons (Int) {'default': None, 'conditions': [], 'min_value': 500, 'max_value': 800, 'step': 20, 'sampling': None}
### Results
Trial 15 Complete [00h 00m 08s]

Best val_loss So Far: 0.8480988144874573
Total elapsed time: 00h 02m 21s
INFO:tensorflow:Oracle triggered exit
Results summary

Trial summary
Hyperparameters:
embedding_neurons: 500
Score: 0.8480988144874573

Trial summary
Hyperparameters:
embedding_neurons: 640
Score: 0.8524157404899597

Trial summary
Hyperparameters:
embedding_neurons: 600
Score: 0.8538790345191956

Trial summary
Hyperparameters:
embedding_neurons: 680
Score: 0.8540633320808411

Trial summary
Hyperparameters:
embedding_neurons: 520
Score: 0.856784999370575













# focusing on num hidden layers trial 1 SELECTED 4
### Search Space
Search space summary
Default search space size: 2
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 310, 'step': 100, 'sampling': None}

### Results

Trial 122 Complete [00h 00m 13s]

Best val_loss So Far: 0.7372345924377441
Total elapsed time: 00h 36m 48s

Search: Running Trial #123

Hyperparameter    |Value             |Best Value So Far 
num_hidden_layers |4                 |3                 
dense0_neurons    |110               |110               
dense1_neurons    |210               |110               
dense2_neurons    |110               |110               
dense3_neurons    |110               |310 



# num hidden layers trial 2
### Search Space
Default search space size: 4
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 4, 'step': 1, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 110, 'max_value': 310, 'step': 100, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 110, 'max_value': 310, 'step': 100, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 110, 'max_value': 310, 'step': 100, 'sampling': None}

### Results


Trial 91 Complete [00h 00m 21s]

Best val_loss So Far: 0.739601731300354
Total elapsed time: 00h 29m 21s

Hyperparameters:
num_hidden_layers: 4
dense0_neurons: 110
dense1_neurons: 210
dense2_neurons: 110
dense3_neurons: 210
Score: 0.739601731300354

Trial summary
Hyperparameters:
num_hidden_layers: 4
dense0_neurons: 110
dense1_neurons: 210
dense2_neurons: 210
dense3_neurons: 110
Score: 0.7468829154968262

Trial summary
Hyperparameters:
num_hidden_layers: 4
dense0_neurons: 110
dense1_neurons: 110
dense2_neurons: 110
dense3_neurons: 210
Score: 0.7488954663276672

Trial summary
Hyperparameters:
num_hidden_layers: 3
dense0_neurons: 110
dense1_neurons: 210
dense2_neurons: 210
dense3_neurons: 310
Score: 0.749832272529602

Trial summary
Hyperparameters:
num_hidden_layers: 3
dense0_neurons: 110
dense1_neurons: 110
dense2_neurons: 210
dense3_neurons: 310
Score: 0.7520040273666382

# Num hidden layers trial 3
### Search Space
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 310, 'step': 100, 'sampling': None}

### Results
Trial 189 Complete [00h 00m 05s]

Best val_loss So Far: 0.7709835171699524
Total elapsed time: 00h 18m 53s

Hyperparameters:
num_hidden_layers: 4
dense0_neurons: 100
dense1_neurons: 200
dense2_neurons: 100
dense3_neurons: 100
dense4_neurons: 100
Score: 0.7709835171699524

Trial summary
Hyperparameters:
num_hidden_layers: 4
dense0_neurons: 200
dense1_neurons: 100
dense2_neurons: 100
dense3_neurons: 200
dense4_neurons: 100
Score: 0.7801269888877869

Trial summary
Hyperparameters:
num_hidden_layers: 4
dense0_neurons: 100
dense1_neurons: 100
dense2_neurons: 200
dense3_neurons: 100
dense4_neurons: 200
Score: 0.7853558659553528

Trial summary
Hyperparameters:
num_hidden_layers: 4
dense0_neurons: 100
dense1_neurons: 200
dense2_neurons: 100
dense3_neurons: 200
dense4_neurons: 200
Score: 0.7891634106636047

Trial summary
Hyperparameters:
num_hidden_layers: 4
dense0_neurons: 200
dense1_neurons: 200
dense2_neurons: 100
dense3_neurons: 100
Score: 0.7953263521194458

Trial summary
Hyperparameters:
num_hidden_layers: 4
dense0_neurons: 100
dense1_neurons: 200
dense2_neurons: 100
dense3_neurons: 100
dense4_neurons: 200
Score: 0.7953308820724487

Trial summary
Hyperparameters:
num_hidden_layers: 4
dense0_neurons: 200
dense1_neurons: 100
dense2_neurons: 100
dense3_neurons: 100
dense4_neurons: 300
Score: 0.8158326745033264











# Num Neurons trial 1

batch size 512
### Search Space
Search space summary
Default search space size: 3
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 200, 'step': 50, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 200, 'step': 50, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 200, 'step': 50, 'sampling': None}
### Results
Trial 25 Complete [00h 00m 06s]

Best val_loss So Far: 0.7383661270141602
Total elapsed time: 00h 02m 28s

Hyperparameters:
dense0_neurons: 200
dense1_neurons: 100
dense2_neurons: 150
Score: 0.7383661270141602


Trial summary
Hyperparameters:
dense0_neurons: 150
dense1_neurons: 100
dense2_neurons: 150
Score: 0.7402308583259583

Trial summary
Hyperparameters:
dense0_neurons: 150
dense1_neurons: 100
dense2_neurons: 100
Score: 0.7419638633728027

Trial summary
Hyperparameters:
dense0_neurons: 50
dense1_neurons: 150
dense2_neurons: 200
Score: 0.7456623911857605

# Num Neurons trial 2
focusing on number of neurons in hidden layers

batch size 512
### Search Space
Default search space size: 3
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 100, 'step': 20, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 80, 'max_value': 160, 'step': 20, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 140, 'max_value': 200, 'step': 20, 'sampling': None}
### Results
Trial 59 Complete [00h 00m 05s]

Best val_loss So Far: 0.7368301749229431
Total elapsed time: 00h 05m 50s

Hyperparameters:
dense0_neurons: 80
dense1_neurons: 120
dense2_neurons: 140
Score: 0.7368301749229431

Trial summary
Hyperparameters:
dense0_neurons: 80
dense1_neurons: 140
dense2_neurons: 180
Score: 0.7396628856658936

Trial summary
Hyperparameters:
dense0_neurons: 80
dense1_neurons: 120
dense2_neurons: 180
Score: 0.7421037554740906

Trial summary
Hyperparameters:
dense0_neurons: 80
dense1_neurons: 100
dense2_neurons: 200
Score: 0.742563784122467

Trial summary
Hyperparameters:
dense0_neurons: 100
dense1_neurons: 100
dense2_neurons: 160
Score: 0.7429311275482178



# Num Neurons trial 3
focusing on number of neurons in hidden layers

### Results

Trial 64 Complete [00h 00m 06s]

Best val_loss So Far: 0.7547465562820435
Total elapsed time: 00h 06m 37s

Search: Running Trial #65

Hyperparameter    |Value             |Best Value So Far 
dense0_neurons    |150               |50                
dense1_neurons    |230               |150               
dense2_neurons    |310               |290 



# Num Neurons trial 4
focusing on number of neurons in hidden layers

### Results

Default search space size: 4
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 25, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 150, 'max_value': 250, 'step': 50, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 260, 'max_value': 340, 'step': 20, 'sampling': None}
{'default': None, 'conditions': [], 'min_value': 320, 'max_value': 360, 'step': 20, 'sampling': None}


Trial 73 Complete [00h 00m 06s]

Best val_loss So Far: 0.8590142726898193
Total elapsed time: 00h 07m 42s

Hyperparameters:
dense0_neurons: 50
dense1_neurons: 150
dense2_neurons: 300
dense3_neurons: 340
Score: 0.8590142726898193

Trial summary
Hyperparameters:
dense0_neurons: 75
dense1_neurons: 200
dense2_neurons: 340
dense3_neurons: 360
Score: 0.8650712966918945

Trial summary
Hyperparameters:
dense0_neurons: 75
dense1_neurons: 150
dense2_neurons: 300
dense3_neurons: 320
Score: 0.8786329030990601

Trial summary
Hyperparameters:
dense0_neurons: 50
dense1_neurons: 150
dense2_neurons: 260
dense3_neurons: 320
Score: 0.878955066204071

Trial summary
Hyperparameters:
dense0_neurons: 50
dense1_neurons: 250
dense2_neurons: 260
dense3_neurons: 340
Score: 0.8804830312728882



# Num Neurons Trial 5
focusing on number of neurons in hidden layers

### Results

Search space summary
Default search space size: 4
dense0_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 50, 'sampling': None}
dense1_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 150, 'max_value': 250, 'step': 50, 'sampling': None}
dense2_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 300, 'max_value': 360, 'step': 20, 'sampling': None}
dense3_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 340, 'max_value': 460, 'step': 20, 'sampling': None}


Trial 94 Complete [00h 00m 06s]

Best val_loss So Far: 0.8065950870513916
Total elapsed time: 00h 08m 50s

Hyperparameters:
dense0_neurons: 50
dense1_neurons: 200
dense2_neurons: 340
dense3_neurons: 400
Score: 0.8065950870513916

Trial summary
Hyperparameters:
dense0_neurons: 50
dense1_neurons: 200
dense2_neurons: 320
dense3_neurons: 440
Score: 0.8106071352958679

Trial summary
Hyperparameters:
dense0_neurons: 50
dense1_neurons: 200
dense2_neurons: 320
dense3_neurons: 400
Score: 0.8306876420974731


# Num Neurons Trial 6
focusing on number of neurons in hidden layers

### Results

Trial 213 Complete [00h 00m 07s]

Best val_loss So Far: 0.8554427027702332
Total elapsed time: 00h 24m 47s

Search: Running Trial #214

Hyperparameter    |Value             |Best Value So Far 
dense0_neurons    |300               |100               
dense1_neurons    |200               |200               
dense2_neurons    |500               |100               
dense3_neurons    |100               |200               
dense4_neurons    |400               |200  

# Num Neurons Trial 7
focusing on number of neurons in hidden layers